In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

In [2]:
import pandas as pd
movies=pd.read_csv('movies_clean_new.csv')
movies

,release_year,title,description,Popularity,Vote_Count,rating,Original_Language,listed_in,Poster_Url,movie_id,tagged_des
0,2021-12-15,Spider-Man: No Way Home,Peter Parker is unmasked and no longer able to...,5083.954,8940,8.3,en,"Action, Adventure, Science Fiction",https://image.tmdb.org/t/p/original/1g0dhYtq4i...,1,1 Peter Parker is unmasked and no longer able ...
1,2022-03-01,The Batman,"In his second year of fighting crime, Batman u...",3827.658,1151,8.1,en,"Crime, Mystery, Thriller",https://image.tmdb.org/t/p/original/74xTEgt7R3...,2,"2 In his second year of fighting crime, Batman..."
2,2022-02-25,No Exit,Stranded at a rest stop in the mountains durin...,2618.087,122,6.3,en,Thriller,https://image.tmdb.org/t/p/original/vDHsLnOWKl...,3,3 Stranded at a rest stop in the mountains dur...
3,2021-11-24,Encanto,"The tale of an extraordinary family, the Madri...",2402.201,5076,7.7,en,"Animation, Comedy, Family, Fantasy",https://image.tmdb.org/t/p/original/4j0PNHkMr5...,4,"4 The tale of an extraordinary family, the Mad..."
4,2021-12-22,The King's Man,As a collection of history's worst tyrants and...,1895.511,1793,7.0,en,"Action, Adventure, Thriller, War",https://image.tmdb.org/t/p/original/aq4Pwv5Xeu...,5,5 As a collection of history's worst tyrants a...
...,...,...,...,...,...,...,...,...,...,...,...
7928,1973-10-15,Badlands,A dramatization of the Starkweather-Fugate kil...,13.357,896,7.6,en,"Drama, Crime",https://image.tmdb.org/t/p/original/z81rBzHNgi...,7929,7929 A dramatization of the Starkweather-Fugat...
7929,2020-10-01,Violent Delights,A female vampire falls in love with a man she ...,13.356,8,3.5,es,Horror,https://image.tmdb.org/t/p/original/4b6HY7rud6...,7930,7930 A female vampire falls in love with a man...
7930,2016-05-06,The Offering,When young and successful reporter Jamie finds...,13.355,94,5.0,en,"Mystery, Thriller, Horror",https://image.tmdb.org/t/p/original/h4uMM1wOhz...,7931,7931 When young and successful reporter Jamie ...
7931,2021-03-31,The United States vs. Billie Holiday,Billie Holiday spent much of her career being ...,13.354,152,6.7,en,"Music, Drama, History",https://image.tmdb.org/t/p/original/vEzkxuE2sJ...,7932,7932 Billie Holiday spent much of her career b...


In [3]:
movies['tagged_des']

0       1 Peter Parker is unmasked and no longer able ...
1       2 In his second year of fighting crime, Batman...
2       3 Stranded at a rest stop in the mountains dur...
3       4 The tale of an extraordinary family, the Mad...
4       5 As a collection of history's worst tyrants a...
                              ...                        
7928    7929 A dramatization of the Starkweather-Fugat...
7929    7930 A female vampire falls in love with a man...
7930    7931 When young and successful reporter Jamie ...
7931    7932 Billie Holiday spent much of her career b...
7932    7933 Documentary style account of a nuclear ho...
Name: tagged_des, Length: 7933, dtype: object

In [4]:
movies['tagged_des'].to_csv('tagged_des_new.txt',
                            index=False,
                            header=False,
                            encoding='utf-8')

In [5]:
raw_document=TextLoader('tagged_des_new.txt', encoding='utf-8').load()
text_splitter=CharacterTextSplitter(chunk_size=1,chunk_overlap=0,separator='\n')
documents=text_splitter.split_documents(raw_document)

Created a chunk of size 262, which is longer than the specified 1
Created a chunk of size 166, which is longer than the specified 1
Created a chunk of size 263, which is longer than the specified 1
Created a chunk of size 524, which is longer than the specified 1
Created a chunk of size 159, which is longer than the specified 1
Created a chunk of size 483, which is longer than the specified 1
Created a chunk of size 226, which is longer than the specified 1
Created a chunk of size 226, which is longer than the specified 1
Created a chunk of size 247, which is longer than the specified 1
Created a chunk of size 334, which is longer than the specified 1
Created a chunk of size 518, which is longer than the specified 1
Created a chunk of size 157, which is longer than the specified 1
Created a chunk of size 170, which is longer than the specified 1
Created a chunk of size 166, which is longer than the specified 1
Created a chunk of size 341, which is longer than the specified 1
Created a 

In [6]:
documents[0]

Document(metadata={'source': 'tagged_des_new.txt'}, page_content='"1 Peter Parker is unmasked and no longer able to separate his normal life from the high-stakes of being a super-hero. When he asks for help from Doctor Strange the stakes become even more dangerous, forcing him to discover what it truly means to be Spider-Man."')

In [8]:
db_movies=Chroma.from_documents(documents,
                               embedding=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2'),)

In [9]:
query='A spiderman movie'
docs=db_movies.similarity_search(query, k=3)
docs

[Document(id='e4a36901-3199-42d1-b62a-548f7080855f', metadata={'source': 'tagged_des_new.txt'}, page_content='"136 After being bitten by a genetically altered spider at Oscorp, nerdy but endearing high school student Peter Parker is endowed with amazing powers to become the superhero known as Spider-Man."'),
 Document(id='796786cb-0f2a-4ca1-b62c-663d631eead4', metadata={'source': 'tagged_des_new.txt'}, page_content='3768 This is an Australian-made parody of Italian action–adventure films of the 60s and 70s. and foreign movies that misappropriated popular American superheroes such as the Japanese TV series “Spider-Man”. (It should be noted that the Japanese Spider-Man was officially sanctioned by Marvel and is considered canon in the Marvel universe) This is an Italian take on the comic book superhero Spider-Man.'),
 Document(id='0e274537-8b5e-4e33-a403-15a0e2dd5d97', metadata={'source': 'tagged_des_new.txt'}, page_content='"156 The seemingly invincible Spider-Man goes up against an all

In [10]:
movies[movies['movie_id']==int(docs[0].page_content.split()[0].strip().replace('"', ''))]

,release_year,title,description,Popularity,Vote_Count,rating,Original_Language,listed_in,Poster_Url,movie_id,tagged_des
135,2002-05-01,Spider-Man,After being bitten by a genetically altered sp...,206.376,15336,7.2,en,"Fantasy, Action",https://image.tmdb.org/t/p/original/gh4cZbhZxy...,136,136 After being bitten by a genetically altere...


In [12]:
def retrieve_similar_movies(
        query:str,
        top_k:int=10,
)-> pd.DataFrame:
    recs=db_movies.similarity_search(query, k=50)

    books_list=[]

    for i in range(0,len(recs)):
        books_list+=[int(recs[i].page_content.strip('"').split()[0])]
    return movies[movies['movie_id'].isin(books_list)].head(top_k)

In [14]:
retrieve_similar_movies('A superhero movie').iloc[0]

release_year                                                2022-03-01
title                                                       The Batman
description          In his second year of fighting crime, Batman u...
Popularity                                                    3827.658
Vote_Count                                                        1151
rating                                                             8.1
Original_Language                                                   en
listed_in                                     Crime, Mystery, Thriller
Poster_Url           https://image.tmdb.org/t/p/original/74xTEgt7R3...
movie_id                                                             2
tagged_des           2 In his second year of fighting crime, Batman...
Name: 1, dtype: object